In [1]:
import json
import requests
import threading
import re
import traceback

URL = 'http://3.84.244.86:8080/enhancer/chain/PurchaseOrder'
FLATTEN_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.flatteringTemp'
CHOOSE_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.Temp'
VALIDATION_URL = 'http://34.80.26.185:8086/PO_Processing/ReArrangingofData'
REGEX_URL = 'http://35.221.160.146:5020/regexResult'
OVERLAP_URL = 'http://35.186.166.22:8082/portal/servlet/service/Poheader.poi'

LABEL = 'http://fise.iks-project.eu/ontology/entity-reference'
TEXT_LABEL = 'http://fise.iks-project.eu/ontology/selected-text'
RELATION_LABEL = 'http://purl.org/dc/terms/relation'
SELECTED_LABEL = 'http://bizlem.io/PurchaseOrderProcessing#'

In [8]:
def middle(value, left, right):
    return value >= left and value <= right

def get_x(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['x']

def get_y(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['y']

def to_float(s):
    s_format = ''.join(re.split('[^\d\.]', s))
    return float(s_format)

def is_float(s):
    s_format = ''.join(re.split(',', s))
    try:
        num = float(s_format)
        return True
    except ValueError:
        return False

In [5]:
folder = 'c30'
path = 'p4materials/' + folder + '.json'
# path = '/home/pexea12/N2000HZ118-19-0149628.json'
with open(path) as f:
    data = json.load(f)
    data_sum = data[0]
    data = data[1:]

In [6]:
def parseJSON(data, x_thres, y_thres):
    response = []
    
    # Build histogram
    y = [ (get_y(obj) + get_y(obj, 2)) // 2 for obj in data ]
    y.sort()

    cur_y = y[0]
    hist_y = { cur_y: cur_y }

    for yc in y:
        if yc == cur_y: 
            continue
        if yc - cur_y > y_thres:
            cur_y = yc
        hist_y[yc] = cur_y

    data.sort(key=lambda obj: hist_y[(get_y(obj) + get_y(obj, 2)) // 2] * 1000000 + get_x(obj))
    

    curLineYCoord = (get_y(data[0]) + get_y(data[0], 2)) // 2
    curXEnd = get_x(data[0], 2)
    curXStart = get_x(data[0])
    line = {
        'y': curLineYCoord,
        'words': [],
    }

    completedWord = data[0]['description']
    characters = set(['+', '-', ';', '-', '\\', "'", '"', '/'])
    open_chars = set(['(', '[', '{'])
    close_chars = set([')', ']', '}'])

    for i in range(1, len(data)):
        curWord = data[i]['description']
        yMid = (get_y(data[i]) + get_y(data[i], 2)) // 2
        xStart = get_x(data[i])


        if hist_y[yMid] <= hist_y[curLineYCoord] + y_thres and hist_y[yMid] >= hist_y[curLineYCoord] - y_thres:
            if xStart <= curXEnd + x_thres \
                or (xStart <= curXEnd + 2 * x_thres and ( \
                    curWord[0] in characters \
                    or curWord[0] in close_chars \
                    or ((curWord[0] in set([',', '.', '%'])) \
                        and '0' <= completedWord[-1] and completedWord[-1] <= '9') \
                    or completedWord[-1] in characters \
                    or completedWord[-1] in open_chars \
                    or (len(completedWord) >= 2 and completedWord[-1] in set([',', '.']) \
                        and '0' <= curWord[0] and curWord[0] <= '9' \
                        and '0' <= completedWord[-2] and completedWord[-2] <= '9'))):
                    completedWord += curWord
                    curXEnd = get_x(data[i], 2)
            else:
                line['words'].append({
                    'word': completedWord,
                    'x1': curXStart,
                    'x2': curXEnd,
                })

                curXEnd = get_x(data[i], 2)
                completedWord = data[i]['description']
                curXStart = get_x(data[i])
        else:
            line['words'].append({
                'word': completedWord,
                'x1': curXStart,
                'x2': curXEnd,
            })

            response.append(line)
            curLineYCoord = (get_y(data[i]) + get_y(data[i], 2)) // 2
            curXStart = get_x(data[i])
            curXEnd = get_x(data[i], 2)

            completedWord = data[i]['description']
            line = {
                'y': curLineYCoord,
                'words': [],
            }

    return response

In [9]:
slist = parseJSON(data, 5, 10)

for line_index, line in enumerate(slist):
    s = ' '.join([ word['word'] for word in line['words'] ])
    
    print(line_index, line['y'], s)
    
max_y = slist[-1]['y']
max_x = 0
for line in slist:
    if len(line['words']) == 0:
        continue
        
    last_word = line['words'][-1]['x2']
    if last_word > max_x:
        max_x = last_word
        
print('max', max_x, max_y)

0 7 Authorized Distributors fora b
1 39 tes A Chicago 26
2 52 Pneumatica
3 106 TITAN ENGINEERING SOLUTIONSINS Casal
4 140 ORIGINALFORRECIPENT
5 163 GSTIN 27AMCPM0963G1Z4
6 201 TAX INVOICE
7 242 INVOICE TO: TAXINVNO. 0115
8 266 Name :ESSAR STEELINDIAUMITED, GSTNO. 24AAACE1741PIZN Date ofissue +29/10/2015
9 290 Address :ESSARSTEEL PLANT State GUJARAT WONO ESTL-DTA/SCC/4400125070
10 314 27THKM SURAT-HAZIRAROAD, StateCode :24 PlaceofWork HAZIRA
11 338 HAZIRA-394270GUJARAT State GUJARAT
12 362 CONTACTPERSON:MRKEYUR SHAH State Code 24
13 387 TranspotationMode :BYPRIVATEVEHICLE
14 411 Name ESSAR STEELINDIALIMITED GSTNO. 26AAACE1741PIZN JMSREFNO. 1262
15 435 Address :ESSARSTEELPLATEPLANT, State GUJARAT Reverse Chargest
16 458 27TH KM SURAT-HAZIRAROAD, StateCode :24
17 483 HAZIRA-394270GUJARAT
18 509 ST Description ofProduct HSN CODE QTY RATE AMT CGST SGST TOTAL(RS)
19 532 DETORQUING& TORQUINGSERVICES 998717 5 2 2.500.00 112,500.00 20,250.00 132,750.00
20 544 OSTS
21 567 2 STANDBYCHARGES 998717

In [10]:
for i, line in enumerate(slist):
    print(i, line['y'], [ word['word'] for word in line['words'] ])

0 7 ['Authorized', 'Distributors', 'fora', 'b']
1 39 ['tes', 'A', 'Chicago', '26']
2 52 ['Pneumatica']
3 106 ['TITAN', 'ENGINEERING', 'SOLUTIONSINS', 'Casal']
4 140 ['ORIGINALFORRECIPENT']
5 163 ['GSTIN', '27AMCPM0963G1Z4']
6 201 ['TAX', 'INVOICE']
7 242 ['INVOICE', 'TO:', 'TAXINVNO.', '0115']
8 266 ['Name', ':ESSAR', 'STEELINDIAUMITED,', 'GSTNO.', '24AAACE1741PIZN', 'Date', 'ofissue', '+29/10/2015']
9 290 ['Address', ':ESSARSTEEL', 'PLANT', 'State', 'GUJARAT', 'WONO', 'ESTL-DTA/SCC/4400125070']
10 314 ['27THKM', 'SURAT-HAZIRAROAD,', 'StateCode', ':24', 'PlaceofWork', 'HAZIRA']
11 338 ['HAZIRA-394270GUJARAT', 'State', 'GUJARAT']
12 362 ['CONTACTPERSON:MRKEYUR', 'SHAH', 'State', 'Code', '24']
13 387 ['TranspotationMode', ':BYPRIVATEVEHICLE']
14 411 ['Name', 'ESSAR', 'STEELINDIALIMITED', 'GSTNO.', '26AAACE1741PIZN', 'JMSREFNO.', '1262']
15 435 ['Address', ':ESSARSTEELPLATEPLANT,', 'State', 'GUJARAT', 'Reverse', 'Chargest']
16 458 ['27TH', 'KM', 'SURAT-HAZIRAROAD,', 'StateCode', ':24']
17

In [38]:
sres = []
sres_words = []

for line in slist:
    sres.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ],
    })
    
    sres_words.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ]
    })
    

def request_line_and_replace(slist, sres, sres_words, line_index):
    sentence = ' '.join([ word for word in sres[line_index]['words'] ])
    res = {}
    try: 
        r = requests.post(URL, data=sentence.encode('utf-8'), headers={'Content-Type': 'application/pdf'})
        r = r.json()

        for obj in r:
            if LABEL in obj and RELATION_LABEL in obj:
                res[obj[LABEL][0]['@id']] = obj[RELATION_LABEL][0]['@id']

        for obj in r:
            if TEXT_LABEL in obj and '@id' in obj:
                obj_id = obj['@id']
                for kurl, vid in res.items():
                    if vid == obj_id:
                        res[kurl] = obj[TEXT_LABEL][0]['@value']
       
        merging_r = {}
        words = slist[line_index]['words']
        
        word_index = 0
        while word_index < len(words):
            word = words[word_index]
            found = False
            
            for kurl, vtext in res.items():
                if kurl in merging_r:
                    continue
                    
                if found:
                    break
                    
                vtext_first_word = vtext.split(' ')[0]
                
                if vtext_first_word not in word['word']:
                    continue
                
                current_word_indexes = [ word_index ]
                current_s = word['word']
                current_i = word_index + 1
                while len(current_s) < len(vtext) and current_i < len(words):
                    current_word_indexes.append(current_i)
                    current_s += ' ' + words[current_i]['word']
                    current_i += 1
                
                if vtext not in current_s:
                    continue
                
                found = True
                merging_r[kurl] = current_word_indexes
                
            if not found:
                word_index += 1
        
#         print(merging_r)

        removed_words = set()
        for url, merging_arr in merging_r.items():
            min_index = merging_arr[0]
            start_word = slist[line_index]['words'][min_index]
            sres_words[line_index]['words'][min_index] = url
            
            for word_index in merging_arr[1:]:
                removed_words.add(word_index)
                current_word = slist[line_index]['words'][word_index]
                start_word['word'] += ' ' + current_word['word']
                start_word['x2'] = current_word['x2']
        
        slist_newarr = []
        sres_words_newarr = []
        
        for word_index, word in enumerate(slist[line_index]['words']):
            if word_index in removed_words: continue
            
            slist_newarr.append(word)
            sres_words_newarr.append(sres_words[line_index]['words'][word_index])
            
        slist[line_index]['words'] = slist_newarr
        sres_words[line_index]['words'] = sres_words_newarr
        
        sres[line_index]['words'] = list(res.keys())
#         print(sres_words_newarr)
                
    except Exception as e:
        traceback.print_exc()
                    
        
def request_all(sres):
    threads = [ threading.Thread(target=request_line_and_replace, args=(slist, sres, sres_words, line_index)) for line_index in range(len(sres)) ]
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

print('requesting...')
request_all(sres)
# request_line_and_replace(slist, sres, sres_words, 1)
print('finish requesting...')

# print(slist[1]['words'])
# print(sres_words[1]['words'])
for line_index, line in enumerate(sres):
    print(line_index, line['y'])
    print('\t', line['words'])
    print('\t', [ word['word'] for word in slist[line_index]['words'] ])
    print('\t', sres_words[line_index]['words'])

requesting...
finish requesting...
0 80
	 ['http://bizlem.io/PurchaseOrderProcessing#AmountNumbers', 'http://bizlem.io/PurchaseOrderProcessing#TaxInvoice']
	 ['TAX INVOICE.-ORIGINALFORRECIPIENT INVOICE', ':5209', '|', 'INVOICEDATE:28/11/2018', '|', 'AMOUNT:10054.00']
	 ['http://bizlem.io/PurchaseOrderProcessing#TaxInvoice', ':5209', '|', 'INVOICEDATE:28/11/2018', '|', 'http://bizlem.io/PurchaseOrderProcessing#AmountNumbers']
1 104
	 []
	 ['CoolLineAgencyPvt.', 'Ltd.', 'Address:']
	 ['CoolLineAgencyPvt.', 'Ltd.', 'Address:']
2 125
	 ['http://bizlem.io/PurchaseOrderProcessing#GSTIN']
	 ['GSTIN', 'No.:', '24AADCC8528K1ZH', '125206', '101-103,', 'ShlokBusinessCenter,BesideAppleHospital,Udhna', 'DarwajaSuratGUJARAT-24']
	 ['http://bizlem.io/PurchaseOrderProcessing#GSTIN', 'No.:', '24AADCC8528K1ZH', '125206', '101-103,', 'ShlokBusinessCenter,BesideAppleHospital,Udhna', 'DarwajaSuratGUJARAT-24']
3 143
	 []
	 ['P.A.N.:', 'AADCC8528K', 'ContactNo.:,70968-11111']
	 ['P.A.N.:', 'AADCC8528K', 'Con

In [39]:
for line_index, line in enumerate(sres_words):
    for word in line['words']:
        if SELECTED_LABEL in word:
            print(line_index, line['y'], word)

0 80 http://bizlem.io/PurchaseOrderProcessing#TaxInvoice
0 80 http://bizlem.io/PurchaseOrderProcessing#AmountNumbers
2 125 http://bizlem.io/PurchaseOrderProcessing#GSTIN
8 234 http://bizlem.io/PurchaseOrderProcessing#Transport
10 272 http://bizlem.io/PurchaseOrderProcessing#Date
12 312 http://bizlem.io/PurchaseOrderProcessing#GSTIN
13 333 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices
13 333 http://bizlem.io/PurchaseOrderProcessing#Quantity
13 333 http://bizlem.io/PurchaseOrderProcessing#Unit
13 333 http://bizlem.io/PurchaseOrderProcessing#Discount
14 354 http://bizlem.io/PurchaseOrderProcessing#AmountNumbers
14 354 http://bizlem.io/PurchaseOrderProcessing#Rate
18 466 http://bizlem.io/PurchaseOrderProcessing#Tax_Code
24 601 http://bizlem.io/PurchaseOrderProcessing#GrossTotal
26 638 http://bizlem.io/PurchaseOrderProcessing#GrossTotal


In [40]:
header_keys = set([
    'SNo',
    'DescriptionOfServices',
    'HSN/SAC',
    'Unit',
    'Quantity',
    'Rate',
    'AmountNumbers',
    'SGST',
    'CGST',
    'IGST',
])


selected = []
header_map_arr = []


end = False

for line_index, words in enumerate(slist):
    if len(sres[line_index]['words']) == 0: continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key not in header_keys: continue
            
        x1, x2 = word['x1'], word['x2']
        
        header_map_arr.append((
            SELECTED_LABEL + url_key,
            line_index,
            words['y'],
            x1,
            x2,
        ))

freq = [0] * max_y

for (url_key, _, y, _, _) in header_map_arr:
    start_y = max(0, y - 30)
    stop_y = min(max_y, y + 30)
    for i in range(start_y, stop_y + 1):
        freq[i] += 1

clusters_top = []
for i in range(1, max_y - 1):
    if freq[i] > freq[i - 1] and freq[i] > freq[i + 1]:
        clusters_top.append((i, freq[i]))
    elif freq[i] > freq[i - 1] and freq[i] == freq[i + 1]:
        for j in range(i + 1, max_y - 1):
            if freq[j] < freq[i]:
                clusters_top.append((i, freq[i]))
                break
            elif freq[j] > freq[i]:
                break

clusters_top.sort(key=lambda x: -x[1])
print(clusters_top)

cluster_index = 0
description_index = None
amount_index = None
cluster_headers = []
max_cluster_headers = []
amount_cluster_headers = []

while cluster_index < len(clusters_top): 
    current_y = clusters_top[cluster_index][0]
    for (url_key, line_index, y, x1, x2) in header_map_arr:
        if y >= current_y - 30 and y <= current_y + 30:
            cluster_headers.append((url_key, line_index, y, x1, x2))
            if 'Description' in url_key:
                description_index = cluster_index
            if 'Amount' in url_key:
                amount_index = cluster_index

    if cluster_index == 0:
        max_cluster_headers = cluster_headers
        
    if amount_index is not None:
        amount_cluster_headers = cluster_headers

    if description_index is not None:
        break
    else:
        cluster_index += 1
        cluster_headers = []
        
if description_index is None:
    if amount_index is not None:
        cluster_headers = amount_cluster_headers
    else:
        cluster_headers = max_cluster_headers
        
main_table_headers = cluster_headers

print(json.dumps(main_table_headers, indent=2))

[(324, 5), (50, 1)]
[
  [
    "http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices",
    13,
    333,
    180,
    237
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Quantity",
    13,
    333,
    427,
    449
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Unit",
    13,
    333,
    479,
    552
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#AmountNumbers",
    14,
    354,
    690,
    728
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Rate",
    14,
    354,
    811,
    831
  ]
]


In [ ]:
print(json.dumps(slist[36], indent=2))

In [41]:
header_line_index = list(set([ line_index for (_, line_index, _, _, _) in main_table_headers ]))[:4]
header_scan_index = [ len(slist[line_index]['words']) - 1 for line_index in header_line_index ]
header_lines = []
header_boxes = []

while True:
    right_most_line = None
    right_x = 0
    for i, line_index in enumerate(header_line_index):
        if header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        if words[header_scan_index[i]]['x2'] > right_x:
            right_x = words[header_scan_index[i]]['x2']
            right_most_line = i
    
    print(header_scan_index, right_most_line)
    is_middle = True
    arr = [ ' ' ] * len(header_line_index)
    
    current_line_index = header_line_index[right_most_line]
    current_word_index = header_scan_index[right_most_line]
    current_word = slist[current_line_index]['words'][current_word_index]
    
    arr[right_most_line] = current_word['word']
    
    left_x = current_word['x1']
    right_x = current_word['x2']
    y = slist[current_line_index]['y']
    
    print('\t', current_word['word'], y, left_x, right_x)
    
    is_middle = True
    for i, line_index in enumerate(header_line_index):
        if i == right_most_line or header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        word_index = header_scan_index[i]
        
        if words[word_index]['x2'] > current_word['x1']:
            arr[i] = words[word_index]['word']
            header_scan_index[i] -= 1
            
            left_x = min(left_x, words[word_index]['x1'])
            right_x = max(right_x, words[word_index]['x2'])
            
            is_middle = False
    
    print('\tis_middle', is_middle)
    if is_middle:
        if current_word_index >= 1:
            prev_word = slist[current_line_index]['words'][current_word_index - 1]
            mid_word_x = (current_word['x2'] + prev_word['x1']) // 2
            
            for i, line_index in enumerate(header_line_index):
                if i >= right_most_line or header_scan_index[i] < 0: continue
                words = slist[line_index]['words']
                word_index = header_scan_index[i]
                
                if words[word_index]['x1'] <= mid_word_x and words[word_index]['x2'] >= mid_word_x:
                    arr[i] = words[word_index]['word']
                    left_x = min(left_x, words[word_index]['x1'])
                    right_x = max(right_x, words[word_index]['x2'])
    
    valid_arr = False
    for word in arr:
        if re.search('[a-zA-Z]+', word) is not None:
            valid_arr = True
            break
            
    if valid_arr:
        header_lines.append(arr)
        header_boxes.append((current_line_index, y, left_x, right_x))
    
    header_scan_index[right_most_line] -= 1
    
    print(arr, y, left_x, right_x)
    
    negative_header_scan_index = [ scan_index for scan_index in header_scan_index if scan_index < 0 ]
    if len(negative_header_scan_index) == len(header_scan_index):
        break
    
    print()

for header_arr in header_lines:
    query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
    print('%30s   %s' % (header_arr, query_param))
    

[9, 10] 1
	 Amount 354 1042 1083
	is_middle False
['1.Gst', 'Amount'] 354 1027 1083

[8, 9] 1
	 Rate% 354 1000 1031
	is_middle True
[' ', 'Rate%'] 354 1000 1031

[8, 8] 1
	 Amount 354 948 988
	is_middle False
['C.Gst', 'Amount'] 354 937 988

[7, 7] 1
	 % 354 934 937
	is_middle True
[' ', '%'] 354 934 937

[7, 6] 1
	 Rate 354 908 928
	is_middle True
[' ', 'Rate'] 354 908 928

[7, 5] 1
	 Amount 354 858 897
	is_middle False
['S.Gst', 'Amount'] 354 840 897

[6, 4] 1
	 % 354 839 842
	is_middle True
[' ', '%'] 354 839 842

[6, 3] 1
	 Rate 354 811 831
	is_middle True
[' ', 'Rate'] 354 811 831

[6, 2] 0
	 Taxable 333 754 789
	is_middle False
['Taxable', 'Value'] 333 754 789

[5, 1] 1
	 Amount 354 690 728
	is_middle False
['Total Discount', 'Amount'] 354 618 728

[4, 0] 1
	 % 354 660 663
	is_middle True
[' ', '%'] 354 660 663

[4, -1] 0
	 Unit Rate 333 479 552
	is_middle True
['Unit Rate', ' '] 333 479 552

[3, -1] 0
	 Qty. 333 427 449
	is_middle True
['Qty.', ' '] 333 427 449

[2, -1] 0
	 Hsn/

In [42]:
def request_header(header_arr, header_index, res_header):
    try:
        query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
        print(query_param)
        r = requests.post(FLATTEN_URL, data=json.dumps(query_param))
        
        r = r.json()
        if len(r['response']['docs']) > 0:
            phrase = r['response']['docs'][0]['url']
            res_header[header_index] = phrase
            return phrase
        else:
            res_header[header_index] = None
            return None
            
    except Exception as e:
        traceback.print_exc()
    
res_header = [ None ] * len(header_lines)
threads = [ threading.Thread(target=request_header, args=(header_arr, header_index, res_header)) for header_index, header_arr in enumerate(header_lines) ]
print(len(threads))

print('requesting...')
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
print('finish requesting...')
    
print(res_header)
print(header_boxes)

13
requesting...
{'H4': 'Amount', 'H3': '1.Gst', 'H2': ' ', 'H1': ' '}
{'H4': 'Rate%', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Amount', 'H3': 'C.Gst', 'H2': ' ', 'H1': ' '}
{'H4': 'Rate', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Amount', 'H3': 'S.Gst', 'H2': ' ', 'H1': ' '}
{'H4': 'Rate', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Value', 'H3': 'Taxable', 'H2': ' ', 'H1': ' '}
{'H4': 'Amount', 'H3': 'Total Discount', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': 'Unit Rate', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': 'Qty.', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': 'Hsn/Sac', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': 'Description', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': 'No.', 'H2': ' ', 'H1': ' '}
finish requesting...
[['http://bizlem.io/PurchaseOrderProcessing#GSTPercentage'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['C.Gst Amount'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['S.Gst Amount'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['http://bizlem.io/PurchaseOrderP

In [43]:
main_res_header = [ arr for arr in res_header if arr is not None ]
main_header_boxes = [ header_boxes[i] for i in range(len(res_header)) if res_header[i] is not None ]
res_header_url = [ None ] * len(main_res_header)

print(main_res_header)
print(main_header_boxes)
print()

def request_chosen_header(main_res_header, res_header_url):
    try:
        header_query = dict([ ('column' + str(len(main_res_header) - i), harr[0]) for i, harr in enumerate(main_res_header) ])
        print(json.dumps(header_query, indent=2))
        r = requests.post(CHOOSE_URL, data=json.dumps(header_query))
#         print('hello', r.text)
        print('json', json.dumps(r.json(), indent=2))
        r = r.json()['response']['docs'][0]['RequiredColumns'][0]
        r = json.loads(r)
        print(json.dumps(r, indent=2))
        
        for col_index, col_url in r.items():
            res_index = int(col_index[6:])
            res_header_url[-res_index] = col_url

        return r
            
    except Exception as e:
        traceback.print_exc()
    
request_chosen_header(main_res_header, res_header_url)
res_header_url

[['http://bizlem.io/PurchaseOrderProcessing#GSTPercentage'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['C.Gst Amount'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['S.Gst Amount'], ['http://bizlem.io/PurchaseOrderProcessing#Rate'], ['http://bizlem.io/PurchaseOrderProcessing#TaxableAmount'], ['http://bizlem.io/PurchaseOrderProcessing#TOTAL'], ['http://bizlem.io/PurchaseOrderProcessing#Unit'], ['http://bizlem.io/PurchaseOrderProcessing#Quantity'], ['http://bizlem.io/PurchaseOrderProcessing#HSN/SAC'], ['http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices'], ['http://bizlem.io/PurchaseOrderProcessing#SRNO']]
[(14, 354, 1027, 1083), (14, 354, 1000, 1031), (14, 354, 937, 988), (14, 354, 908, 928), (14, 354, 840, 897), (14, 354, 811, 831), (13, 333, 754, 789), (14, 354, 618, 728), (13, 333, 479, 552), (13, 333, 427, 449), (13, 333, 359, 398), (13, 333, 180, 237), (13, 333, 48, 65)]

{
  "column13": "http://bizlem.io/PurchaseOrderProcessing#GSTPercentage",
  "colum

['http://bizlem.io/PurchaseOrderProcessing#GSTPercentage',
 'http://bizlem.io/PurchaseOrderProcessing#Rate',
 'C.Gst Amount',
 'http://bizlem.io/PurchaseOrderProcessing#Rate',
 'S.Gst Amount',
 'http://bizlem.io/PurchaseOrderProcessing#Rate',
 'http://bizlem.io/PurchaseOrderProcessing#TaxableAmount',
 'http://bizlem.io/PurchaseOrderProcessing#TOTAL',
 'http://bizlem.io/PurchaseOrderProcessing#Unit',
 'http://bizlem.io/PurchaseOrderProcessing#Quantity',
 'http://bizlem.io/PurchaseOrderProcessing#HSN/SAC',
 'http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices',
 'http://bizlem.io/PurchaseOrderProcessing#SRNO']

In [44]:
for i, header in enumerate(res_header_url):
    print(i, header, main_header_boxes[i])
    
print(len(main_header_boxes), len(res_header_url), len(main_res_header))

0 http://bizlem.io/PurchaseOrderProcessing#GSTPercentage (14, 354, 1027, 1083)
1 http://bizlem.io/PurchaseOrderProcessing#Rate (14, 354, 1000, 1031)
2 C.Gst Amount (14, 354, 937, 988)
3 http://bizlem.io/PurchaseOrderProcessing#Rate (14, 354, 908, 928)
4 S.Gst Amount (14, 354, 840, 897)
5 http://bizlem.io/PurchaseOrderProcessing#Rate (14, 354, 811, 831)
6 http://bizlem.io/PurchaseOrderProcessing#TaxableAmount (13, 333, 754, 789)
7 http://bizlem.io/PurchaseOrderProcessing#TOTAL (14, 354, 618, 728)
8 http://bizlem.io/PurchaseOrderProcessing#Unit (13, 333, 479, 552)
9 http://bizlem.io/PurchaseOrderProcessing#Quantity (13, 333, 427, 449)
10 http://bizlem.io/PurchaseOrderProcessing#HSN/SAC (13, 333, 359, 398)
11 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices (13, 333, 180, 237)
12 http://bizlem.io/PurchaseOrderProcessing#SRNO (13, 333, 48, 65)
13 13 13


In [14]:
header_transform = []
count = len(res_header_url)
for i, header in enumerate(res_header_url):
    title = 'a' + str(count)
    line_index, y, x1, x2 = main_header_boxes[i]
    
    header_transform.append({
        title: title,
        title + '_line_index': line_index,
        title + '_url': header if SELECTED_LABEL in header else None,
        title + '_url_found': True if SELECTED_LABEL in header else False,
        title + '_word': header,
        title + '_x1': x1,
        title + '_x2': x2,
        title + '_y': y,
    })
    
    count -= 1
    
print(header_transform)

[{'a11': 'a11', 'a11_line_index': 22, 'a11_url': 'http://bizlem.io/PurchaseOrderProcessing#AMOUNT', 'a11_url_found': True, 'a11_word': 'http://bizlem.io/PurchaseOrderProcessing#AMOUNT', 'a11_x1': 2321, 'a11_x2': 2381, 'a11_y': 1152}, {'a11': 'a11', 'a11_line_index': 22, 'a11_url': 'http://bizlem.io/PurchaseOrderProcessing#TOTAL', 'a11_url_found': True, 'a11_word': 'http://bizlem.io/PurchaseOrderProcessing#TOTAL', 'a11_x1': 2232, 'a11_x2': 2308, 'a11_y': 1152}, {'a11': 'a11', 'a11_line_index': 20, 'a11_url': 'http://bizlem.io/PurchaseOrderProcessing#SGST', 'a11_url_found': True, 'a11_word': 'http://bizlem.io/PurchaseOrderProcessing#SGST', 'a11_x1': 2032, 'a11_x2': 2102, 'a11_y': 1127}, {'a11': 'a11', 'a11_line_index': 20, 'a11_url': 'http://bizlem.io/PurchaseOrderProcessing#CGST', 'a11_url_found': True, 'a11_word': 'http://bizlem.io/PurchaseOrderProcessing#CGST', 'a11_x1': 1771, 'a11_x2': 1841, 'a11_y': 1127}, {'a11': 'a11', 'a11_line_index': 20, 'a11_url': 'http://bizlem.io/PurchaseOrd

In [45]:
header_pos = []
start_line_index = 0

for i, (line_index, y, left_x, right_x) in enumerate(main_header_boxes):
    if res_header_url[i] is None: continue
    header = res_header_url[i]
    header_pos.append((header, line_index, y, [left_x, right_x]))
    if start_line_index < line_index:
        start_line_index = line_index
    
header_pos.sort(key=lambda x: x[-1][0])

stop_line_index = start_line_index + 1

end = False
for line_index, words in enumerate(slist):
    if line_index <= stop_line_index or len(sres[line_index]['words']) == 0: 
        continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key == 'GrossTotal' or 'SGST' in url_key or 'CGST' in url_key or 'IGST' in url_key: 
            stop_line_index = line_index
            end = True
            break
    if end:
        break

# print(json.dumps(header_pos, indent=2))
print(start_line_index, stop_line_index)
print(header_pos)

14 24
[('http://bizlem.io/PurchaseOrderProcessing#SRNO', 13, 333, [48, 65]), ('http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices', 13, 333, [180, 237]), ('http://bizlem.io/PurchaseOrderProcessing#HSN/SAC', 13, 333, [359, 398]), ('http://bizlem.io/PurchaseOrderProcessing#Quantity', 13, 333, [427, 449]), ('http://bizlem.io/PurchaseOrderProcessing#Unit', 13, 333, [479, 552]), ('http://bizlem.io/PurchaseOrderProcessing#TOTAL', 14, 354, [618, 728]), ('http://bizlem.io/PurchaseOrderProcessing#TaxableAmount', 13, 333, [754, 789]), ('http://bizlem.io/PurchaseOrderProcessing#Rate', 14, 354, [811, 831]), ('S.Gst Amount', 14, 354, [840, 897]), ('http://bizlem.io/PurchaseOrderProcessing#Rate', 14, 354, [908, 928]), ('C.Gst Amount', 14, 354, [937, 988]), ('http://bizlem.io/PurchaseOrderProcessing#Rate', 14, 354, [1000, 1031]), ('http://bizlem.io/PurchaseOrderProcessing#GSTPercentage', 14, 354, [1027, 1083])]


In [23]:
items_all = []

def request_overlap(word, header_data, container):
    try:
        header, header_line_index, header_y, header_pos = header_data
        r = requests.post(OVERLAP_URL, data={
            'user_name': 'carrotrule_xyz.com',
            'project_name': 'DataExtractionPO-2',
            'Rule_Engine': 'RuleData-2',
            'RawJson': {
                'wX1': word['x1'],
                'wX2': word['x2'],
                'hX1': header_pos[0],
                'hX2': header_pos[1],
            },
        })
        
        r = r.json()
        
        container.append({
            'header': header if r['OVERLAP'] else '',
            'data': word['word'],
            'data_type': 'number' if is_float(word['word']) else 'string',
            'header_x1': header_pos[0],
            'header_x2': header_pos[1],
            'header_y': header_y,
            'data_x1': word['x1'],
            'data_x2': word['x2'],
            'data_y': word['y'],
        })
        
    except Exception as e:
        traceback.print_exc()
        
def overlap(word_pos, header_pos):
    return word_pos[1] >= header_pos[0] and word_pos[0] <= header_pos[1]

def find_header(word_pos, header_pos):
    for i, (_, _, _, hpos) in enumerate(header_pos):
        if overlap(word_pos, hpos):
            return i
    return None

def create_table_item(type='Data no header'):
    return {
        'type': type,
        'header': '',
        'header_x1': '',
        'header_x2': '',
        'header_y': '',
        'data': '',
        'line_number': '',
        'data_type': '',
        'data_x1': '',
        'data_x2': '',
        'data_y': '',
    }
    

table_data = []
last_header_index = None
for line_index, words in enumerate(slist[start_line_index + 1:stop_line_index]):
    for word in words['words']:
        header_index = find_header((word['x1'], word['x2']), header_pos)
        table_item = create_table_item()
        table_item['data'] = word['word']
        table_item['line_number'] = line_index + (start_line_index + 1)
        table_item['data_type'] = 'number' if is_float(word['word']) else 'string'
        table_item['data_x1'] = word['x1']
        table_item['data_x2'] = word['x2']
        table_item['data_y'] = words['y']
        
        if header_index is not None:
            stop_header_index = header_index + len(header_pos)
            # not consecutive header
            if last_header_index is not None and (stop_header_index - last_header_index) % len(header_pos) > 1: 
                for i in range(last_header_index, stop_header_index):
                    no_data_item = create_table_item(type='Header no data')
                    current_header, _, current_header_y, current_header_x = header_pos[i % len(header_pos)]
                    no_data_item['header'] = current_header
                    no_data_item['header_y'] = current_header_y
                    no_data_item['header_x1'] = current_header_x[0]
                    no_data_item['header_x2'] = current_header_x[1]
                    table_data.append(no_data_item)
                
            
            last_header_index = header_index
            header, header_line_index, header_y, (header_x1, header_x2) = header_pos[header_index]
            
            table_item['type'] = 'Overlap'
            table_item['header_x1'] = header_x1
            table_item['header_x2'] = header_x2
            table_item['header_y'] = header_y
            table_item['header'] = header
            
        table_data.append(table_item)
        
print(json.dumps(table_data, indent=2))

[
  {
    "type": "Overlap",
    "header": "http://bizlem.io/PurchaseOrderProcessing#TaxableAmount",
    "header_x1": 1509,
    "header_x2": 1610,
    "header_y": 1127,
    "data": "value",
    "line_number": 23,
    "data_type": "string",
    "data_x1": 1523,
    "data_x2": 1600,
    "data_y": 1172
  },
  {
    "type": "Data no header",
    "header": "",
    "header_x1": "",
    "header_x2": "",
    "header_y": "",
    "data": ".",
    "line_number": 24,
    "data_type": "string",
    "data_x1": 215,
    "data_x2": 213,
    "data_y": 1195
  },
  {
    "type": "Data no header",
    "header": "",
    "header_x1": "",
    "header_x2": "",
    "header_y": "",
    "data": "R\u00e1t\u00e9",
    "line_number": 25,
    "data_type": "string",
    "data_x1": 1666,
    "data_x2": 1729,
    "data_y": 1206
  },
  {
    "type": "Overlap",
    "header": "http://bizlem.io/PurchaseOrderProcessing#CGST",
    "header_x1": 1771,
    "header_x2": 1841,
    "header_y": 1127,
    "data": "Amt.",
    "line_n

In [16]:
footer_headers = {
    'GrossTotal': number_regex,
    'TOTAL': number_regex,
    'SGSTAmt': number_regex,
    'CGSTAmt': number_regex,
    'IGSTAmt': number_regex,
    'TotalAmt': number_regex,
    'CGSTAmount': number_regex,
    'SGSTAmount': number_regex,
    'IGSTAmount': number_regex,
    'CGST': percentage_regex,
    'SGST': percentage_regex,
    'IGST': percentage_regex,
}

footer_lines = []
for lindex, line in enumerate(slist[stop_line_index:]):
    line_index = lindex + stop_line_index
    item = {}
    for word_index, words in enumerate(line['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL in url:
            term = url.split('#')[-1]
            if term in footer_headers:
                paragraph = concat(slist, line_index, word_index, n_lines=0)
                regex = footer_headers[term]
                search_p = re.findall(number_regex, paragraph)
                if len(search_p) > 0:
                    item[url] = [ to_float(i) for i in search_p ]
    
    if len(item.keys()) == 0:
        continue
        
    item['line_no'] = line_index
    footer_lines.append(item)
    

    

In [17]:

table_output = {
    'header_table_only_url': items_url,
    'header_table_all': items_all,
    'footer_table_only_url': footer_lines,
}

print(json.dumps(table_output, indent=2))

{
  "header_table_only_url": [
    {
      "http://bizlem.io/PurchaseOrderProcessing#SRNO": "",
      "http://bizlem.io/PurchaseOrderProcessing#Description": "Services ",
      "http://bizlem.io/PurchaseOrderProcessing#HSN/SAC": "",
      "http://bizlem.io/PurchaseOrderProcessing#Quantity": "",
      "http://bizlem.io/PurchaseOrderProcessing#Rate": "",
      "http://bizlem.io/PurchaseOrderProcessing#Unit": "",
      "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "",
      "http://bizlem.io/PurchaseOrderProcessing#PreTaxAmount": "",
      "http://bizlem.io/PurchaseOrderProcessing#SGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#CGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#IGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#SGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#CGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#IGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#TOTAL": "",
      "http://

In [32]:
def request_validation(items):
#     try:
        query_params = {
            'header_table': items,
            'footer_table': []
        }
        
        r = requests.post(VALIDATION_URL, data=json.dumps(query_param))
        
        r = r.text
        print(r)
            
#     except Exception as e:
#         print(items, 'error request_validation')
#         print(e)
    
request_validation(items)

NameError: name 'items' is not defined

In [ ]:
slist

# Regex for Invoice Number 

In [20]:
def concat_rows_and_cols(slist, line_index, word_index, n_lines=10):
    paragraph = []
    for word in slist[line_index]['words'][word_index:]:
        nword = word.copy()
        nword['y'] = slist[line_index]['y']
        nword['line_index'] = line_index
        paragraph.append(nword)
    
    current_word = slist[line_index]['words'][word_index]
    
    for lindex in range(line_index + 1, min(line_index + n_lines + 1, len(slist))):
        line = slist[lindex]
        for word in line['words']:
            if word['x1'] >= current_word['x1'] - 15 and word['x1'] <= current_word['x2']:
                nword = word.copy()
                nword['y'] = line['y']
                nword['line_index'] = lindex
                paragraph.append(nword)
    
    return paragraph

def request_regex_word(word, word_index, container):
    try:
        r = requests.post(REGEX_URL, data=word['word'])
        container[word_index] = r.json()
    except Exception as e:
        traceback.print_exc()

def request_regex_header(slist, line_index, word_index, word_url, paragraph, regex_res):
    container = [ None ] * len(paragraph)
    threads = [ threading.Thread(target=request_regex_word, args=(word, word_index, container)) for word_index, word in enumerate(paragraph) ]
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
        
    print(container[1], paragraph[1])

    regex_res.setdefault(word_url, {
        'Regex': [],
        'Integer': [],
        'Date': [],
        'UKS': [],
    })
    
    current_word = slist[line_index]['words'][word_index]
    for rr_index, rr in enumerate(container):
        word = paragraph[rr_index]
        for item in rr:
            regex_res[word_url]['Regex'].append({
                'Regex_Type': item['Regex_Type'],
                'Regex_Value': item['value'],
                'Regex_Id': item['Regex_id'],
                'Regex_Length': item['length'],
                'Regex_Line_OffSet': word['line_index'] - line_index,
                'Regex_X1 OffSet': word['x1'] - current_word['x1'],
                'Regex_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Regex_Y OffSet': word['y'] - slist[line_index]['y'],
                'Regex_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Regex_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            
    for word in paragraph:
        search_result = re.search('\d+', word['word'])
        if search_result is not None:
            regex_res[word_url]['Integer'].append({
                'Integer_Value': word['word'][search_result.start():search_result.end()],
                'Integer_Length': search_result.end() - search_result.start(),
                'Integer_Line_OffSet': word['line_index'] - line_index,
                'Integer_X1 OffSet': word['x1'] - current_word['x1'],
                'Integer_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Integer_Y OffSet': word['y'] - slist[line_index]['y'],
                'Integer_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Integer_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            
        search_result_date = re.search(date_regex, word['word'])
        if search_result_date is not None:
            regex_res[word_url]['Date'].append({
                'Date_Value': word['word'][search_result_date.start():search_result_date.end()],
                'Date_Length': search_result_date.end() - search_result_date.start(),
                'Date_Line_OffSet': word['line_index'] - line_index,
                'Date_X1 OffSet': word['x1'] - current_word['x1'],
                'Date_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Date_Y OffSet': word['y'] - slist[line_index]['y'],
                'Date_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Date_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })

paragraph = concat_rows_and_cols(slist, 65, 1)
print(paragraph)
# regex_res = {}
# request_regex_header(slist, 65, 1, sres_words[65]['words'][1], paragraph, regex_res)
# regex_res

[{'word': '&', 'x1': 1286, 'x2': 1296, 'y': 2687, 'line_index': 65}, {'word': 'IFS', 'x1': 1315, 'x2': 1365, 'y': 2687, 'line_index': 65}, {'word': 'Code', 'x1': 1384, 'x2': 1467, 'y': 2687, 'line_index': 65}, {'word': ':', 'x1': 1487, 'x2': 1497, 'y': 2687, 'line_index': 65}, {'word': 'Vidhyutnagar', 'x1': 1527, 'x2': 1758, 'y': 2687, 'line_index': 65}]


IndexError: list index out of range

In [ ]:
sres_words[8]

In [48]:
regex_urls = [
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Date",
    "http://bizlem.io/PurchaseOrderProcessing#PAN_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number",
    "http://bizlem.io/PurchaseOrderProcessing#TAN",
    "http://bizlem.io/PurchaseOrderProcessing#GSTIN",
    "http://bizlem.io/PurchaseOrderProcessing#TRNO",
    "http://bizlem.io/PurchaseOrderProcessing#TRDate",
    "http://bizlem.io/PurchaseOrderProcessing#CIN",
    "http://bizlem.io/PurchaseOrderProcessing#VAT",
    "http://bizlem.io/PurchaseOrderProcessing#PODate",
    "http://bizlem.io/PurchaseOrderProcessing#Date",
]



threads = []
regex_res = {}
for line_index, line in enumerate(slist):
#     print(line)
    for word_index, word in enumerate(line['words']):
#         print('\t', line_index, word_index)
        word_url = sres_words[line_index]['words'][word_index]
        if word_url in regex_urls:
            print(line_index, word_index, word_url)
            paragraph = concat_rows_and_cols(slist, line_index, word_index)
            
            threads.append(threading.Thread(
                target=request_regex_header, 
                args=(slist, line_index, word_index, word_url, paragraph, regex_res),
            ))
            
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
    
print(json.dumps(regex_res, indent=2))
            

1 3 http://bizlem.io/PurchaseOrderProcessing#Invoice_Number
1 5 http://bizlem.io/PurchaseOrderProcessing#Date
8 0 http://bizlem.io/PurchaseOrderProcessing#GSTIN
15 0 http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number
15 2 http://bizlem.io/PurchaseOrderProcessing#Date
19 6 http://bizlem.io/PurchaseOrderProcessing#Date
22 0 http://bizlem.io/PurchaseOrderProcessing#GSTIN
30 1 http://bizlem.io/PurchaseOrderProcessing#Invoice_Number
67 1 http://bizlem.io/PurchaseOrderProcessing#PAN_Number
[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_5', 'length': 5, 'value': '10040'}] {'word': '10040', 'x1': 1182, 'x2': 1282, 'y': 374, 'line_index': 8}
[] {'word': ':', 'x1': 570, 'x2': 581, 'y': 2716, 'line_index': 67}
[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_4', 'length': 4, 'value': '2017'}, {'Regex_Type': 'DateFormat', 'Regex_id': 'dt_28', 'length': 11, 'value': '17-Jan-2017'}] {'word': '17-Jan-2017', 'x1': 1688, 'x2': 1908, 'y': 577, 'line_index': 17}
[] {'word': '.:', 'x1': 1483, 'x2': 1504, 